In [1]:
using Revise

using BilevelTrajOpt
using ForwardDiff
using DiffResults

┌ Info: Precompiling BilevelTrajOpt [9688c538-179f-11e9-3174-495cea6b7f67]
└ @ Base loading.jl:1187


In [2]:
num_x = 5
num_y = 5
num_z = num_x + num_y
xymin = -10.
xymax = 10.

fx = x -> x'*x
hx = x -> [x[1] - 1.]
gx = x -> [x[2] - 3., 3. - x[3]]
hxy = (x,y) -> [x[4]*y[1]]
gxy = (x,y) -> [x[5]^2*y[2]^3 - 1., y[4]^2*x[1] + 1]
hy = y -> [y[4] - 2.5]
gy = y -> [2. - y[5]]

#15 (generic function with 1 method)

In [3]:
function full_prob(z)
    x = z[1:num_x]
    y = z[num_x+1:num_x+num_y]
    
    J = fx(x)
    c = vcat(gx(x),gxy(x,y),gy(y))
    ceq = vcat(hx(x),hxy(x,y),hy(y))
    
    gJ = ForwardDiff.gradient(s -> fx(s[1:num_x]), z)
    gc = vcat(ForwardDiff.jacobian(s -> gx(s[1:num_x]), z), 
              ForwardDiff.jacobian(s -> gxy(s[1:num_x],s[num_x+1:num_x+num_y]),z),
              ForwardDiff.jacobian(s -> gy(s[num_x+1:num_x+num_y]), z))
    gceq = vcat(ForwardDiff.jacobian(s -> hx(s[1:num_x]), z), 
                ForwardDiff.jacobian(s -> hxy(s[1:num_x],s[num_x+1:num_x+num_y]),z),
                ForwardDiff.jacobian(s -> hy(s[num_x+1:num_x+num_y]), z))
    
    fail = false
    
    J, c, ceq, gJ, gc, gceq, fail
end

z0 = zeros(num_z)
lb = xymin * ones(num_z)
ub = xymax * ones(num_z)
options = Dict{String, Any}()
options["Derivative option"] = 1
options["Verify level"] = 1
options["Major optimality tolerance"] = 1e-6

zopt, fopt, info = snopt(full_prob, z0, lb, ub, options)

display(info)
display(zopt)

"The problem appears to be infeasible: nonlinear infeasibilities minimized"

10-element Array{Float64,1}:
 -0.16              
  0.0               
  3.0               
  0.0               
  0.0               
  0.0               
  0.0               
  0.0               
  2.4999999999999996
  2.083866328076683 

In [4]:
function F(x)
    fy = y -> y'*y
    hxyy = y -> vcat(hxy(x,y),hy(y))
    gxyy = y -> vcat(gxy(x,y),gy(y),y .- xymax,xymin .- y)
    y0 = zeros(num_y)
    λ0 = zeros(length(hxyy(y0)))
    μ0 = zeros(length(gxyy(y0)))
    ysol,λsol,μsol,csol = auglag_solve(y0,λ0,μ0,fy,hxyy,gxyy)
    
    J = fx(x)
    c = vcat(gx(x),gxy(x,ysol))
    ceq = vcat(hx(x),hxy(x,ysol))
    
    vcat(J,c,ceq)
end

Fres = DiffResults.JacobianResult(F(zeros(num_x)),zeros(num_x))
num_g = length(gx(zeros(num_x))) + length(gxy(zeros(num_x),zeros(num_y)))
num_h = length(hx(zeros(num_x))) + length(hxy(zeros(num_x),zeros(num_y)))
function bilevel_prob(x)
    ForwardDiff.jacobian!(Fres, F, x)
    Fv = DiffResults.value(Fres)
    FJ = DiffResults.jacobian(Fres)
    
    fail = false
    
    J = Fv[1]
    c = Fv[2:1+num_g]
    ceq = Fv[1+num_g+1:1+num_g+num_h]

    gJ = FJ[1,:]
    gc = FJ[2:1+num_g,:]
    gceq = FJ[1+num_g+1:1+num_g+num_h,:]
    
    J, c, ceq, gJ, gc, gceq, fail
end

x0 = zeros(num_x)
lb = xymin * ones(num_x)
ub = xymax * ones(num_x)
options = Dict{String, Any}()
options["Derivative option"] = 1
options["Verify level"] = -1
options["Major optimality tolerance"] = 1e-3

xopt, fopt, info = snopt(bilevel_prob, x0, lb, ub, options)

display(info)
display(xopt)

"The problem appears to be infeasible: nonlinear infeasibilities minimized"

5-element Array{Float64,1}:
 -0.16000007259081067   
  0.0                   
  3.0                   
 -0.00015368556375988624
  0.0                   

In [93]:
fy = y -> y'*y
hxyy = y -> vcat(hxy(xopt,y),hy(y))
gxyy = y -> vcat(gxy(xopt,y),gy(y),y .- xymax,xymin .- y)
y0 = zeros(num_y)
λ0 = zeros(length(hxyy(y0)))
μ0 = zeros(length(gxyy(y0)))
ysol,λsol,μsol,csol = auglag_solve(y0,λ0,μ0,fy,hxyy,gxyy)
display(ysol)

5-element Array{Float64,1}:
 -4.9480167946029995e-15
 -3.071821602792873e-14 
 -9.773466698630485e-15 
  2.5000000000000084    
  2.000000000000007     